# Week 1, Lab 4: Advanced Search & Real-World Applications

## Welcome to the Final Lab!

Congratulations on making it to Lab 4! You've learned search algorithms from scratch. Now let's see how professionals use these concepts with real libraries and real-world problems.

### What You'll Learn

- Using NetworkX for graph algorithms
- Real-world graph problems
- Bidirectional search
- Search in continuous spaces
- Performance optimization tips
- Putting it all together

### Real-World Applications

- Social network analysis
- Route planning on real maps
- Network routing and optimization
- Recommendation systems
- Knowledge graphs

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from typing import List, Tuple, Dict, Set, Optional
from collections import deque
from heapq import heappush, heappop
import time

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print(f"NetworkX version: {nx.__version__}")

## 1. Introduction to NetworkX

NetworkX is the standard Python library for working with graphs and networks.

### Key Concepts:
- **Graph**: Collection of nodes (vertices) and edges
- **Directed vs Undirected**: Edges have direction or not
- **Weighted**: Edges have costs/weights

Let's build a simple city road network!

In [ ]:
# Create a weighted graph representing a city
G = nx.Graph()

# Add locations (nodes)
locations = [
    'Home', 'Office', 'Gym', 'Cafe', 'Park', 
    'Store', 'School', 'Library', 'Hospital'
]
G.add_nodes_from(locations)

# Add roads (edges) with travel times in minutes
roads = [
    ('Home', 'Office', 15),
    ('Home', 'Store', 5),
    ('Home', 'School', 10),
    ('Office', 'Cafe', 3),
    ('Office', 'Gym', 8),
    ('Cafe', 'Park', 7),
    ('Gym', 'Park', 5),
    ('Park', 'Hospital', 12),
    ('Store', 'School', 6),
    ('School', 'Library', 4),
    ('Library', 'Hospital', 8),
    ('Cafe', 'Library', 10),
]

for loc1, loc2, time in roads:
    G.add_edge(loc1, loc2, weight=time)

# Visualize the network
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, seed=42, k=2)

# Draw the graph
nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                       node_size=3000, alpha=0.9)
nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
nx.draw_networkx_edges(G, pos, width=2, alpha=0.5)

# Draw edge labels (weights)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=9)

plt.title('City Road Network (weights = travel time in minutes)', 
         fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Network has {G.number_of_nodes()} locations and {G.number_of_edges()} roads")

## 2. Using Built-in Search Algorithms

NetworkX provides optimized implementations of search algorithms.

In [ ]:
# Find shortest path from Home to Hospital
start = 'Home'
goal = 'Hospital'

print(f"Finding route from {start} to {goal}...\n")

# Method 1: Dijkstra's algorithm (similar to UCS)
dijkstra_path = nx.dijkstra_path(G, start, goal, weight='weight')
dijkstra_length = nx.dijkstra_path_length(G, start, goal, weight='weight')

print(f"Dijkstra's Algorithm:")
print(f"  Path: {' → '.join(dijkstra_path)}")
print(f"  Total time: {dijkstra_length} minutes\n")

# Method 2: A* algorithm (needs heuristic)
# For this example, we'll use a simple heuristic
def simple_heuristic(n1, n2):
    """Simple heuristic: return 0 (makes A* behave like Dijkstra)."""
    return 0

astar_path = nx.astar_path(G, start, goal, heuristic=simple_heuristic, weight='weight')
astar_length = nx.astar_path_length(G, start, goal, heuristic=simple_heuristic, weight='weight')

print(f"A* Algorithm:")
print(f"  Path: {' → '.join(astar_path)}")
print(f"  Total time: {astar_length} minutes\n")

# Method 3: All shortest paths
all_paths = list(nx.all_shortest_paths(G, start, goal, weight='weight'))
print(f"All optimal paths ({len(all_paths)} found):")
for i, path in enumerate(all_paths, 1):
    print(f"  {i}. {' → '.join(path)}")

In [ ]:
# Visualize the shortest path
plt.figure(figsize=(14, 10))

# Draw all edges in gray
nx.draw_networkx_nodes(G, pos, node_color='lightgray', 
                       node_size=3000, alpha=0.6)
nx.draw_networkx_edges(G, pos, width=2, alpha=0.2, edge_color='gray')

# Highlight the shortest path
path_edges = [(dijkstra_path[i], dijkstra_path[i+1]) 
              for i in range(len(dijkstra_path)-1)]
nx.draw_networkx_edges(G, pos, edgelist=path_edges, 
                       width=4, edge_color='red', alpha=0.8)

# Highlight start and goal
nx.draw_networkx_nodes(G, pos, nodelist=[start], 
                       node_color='lightgreen', node_size=3500)
nx.draw_networkx_nodes(G, pos, nodelist=[goal], 
                       node_color='lightcoral', node_size=3500)

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=9)

plt.title(f'Shortest Path: {start} → {goal} ({dijkstra_length} min)',
         fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## 3. Bidirectional Search

Search from **both** start and goal simultaneously!

### Why Bidirectional?
- Regular search: Explores O(b^d) nodes
- Bidirectional: Each side explores O(b^(d/2)) nodes
- **Total: 2 × O(b^(d/2)) << O(b^d)**

Example: If b=10, d=6:
- Regular: 1,000,000 nodes
- Bidirectional: 2 × 1,000 = 2,000 nodes
- **500x faster!**

In [ ]:
def bidirectional_search(graph, start, goal):
    """
    Bidirectional BFS - search from both ends.
    
    Returns:
        (path, nodes_explored_forward, nodes_explored_backward)
    """
    if start == goal:
        return [start], 0, 0
    
    # Two frontiers
    forward_queue = deque([start])
    backward_queue = deque([goal])
    
    # Track visited nodes and parents
    forward_visited = {start: None}
    backward_visited = {goal: None}
    
    forward_explored = 0
    backward_explored = 0
    
    while forward_queue and backward_queue:
        # Expand from start
        if forward_queue:
            current_forward = forward_queue.popleft()
            forward_explored += 1
            
            # Check if we met the backward search
            if current_forward in backward_visited:
                # Found meeting point!
                path = reconstruct_bidirectional_path(
                    current_forward, forward_visited, backward_visited
                )
                return path, forward_explored, backward_explored
            
            # Expand forward
            for neighbor in graph.neighbors(current_forward):
                if neighbor not in forward_visited:
                    forward_visited[neighbor] = current_forward
                    forward_queue.append(neighbor)
        
        # Expand from goal
        if backward_queue:
            current_backward = backward_queue.popleft()
            backward_explored += 1
            
            # Check if we met the forward search
            if current_backward in forward_visited:
                path = reconstruct_bidirectional_path(
                    current_backward, forward_visited, backward_visited
                )
                return path, forward_explored, backward_explored
            
            # Expand backward
            for neighbor in graph.neighbors(current_backward):
                if neighbor not in backward_visited:
                    backward_visited[neighbor] = current_backward
                    backward_queue.append(neighbor)
    
    return None, forward_explored, backward_explored

def reconstruct_bidirectional_path(meeting_point, forward_visited, backward_visited):
    """Reconstruct path from bidirectional search."""
    # Build forward path (start to meeting point)
    forward_path = []
    current = meeting_point
    while current is not None:
        forward_path.append(current)
        current = forward_visited[current]
    forward_path.reverse()
    
    # Build backward path (meeting point to goal)
    backward_path = []
    current = backward_visited[meeting_point]
    while current is not None:
        backward_path.append(current)
        current = backward_visited[current]
    
    return forward_path + backward_path

# Test bidirectional search
bidir_path, forward_nodes, backward_nodes = bidirectional_search(G, start, goal)

print("Bidirectional Search Results:")
print(f"  Path: {' → '.join(bidir_path)}")
print(f"  Nodes explored from start: {forward_nodes}")
print(f"  Nodes explored from goal: {backward_nodes}")
print(f"  Total nodes: {forward_nodes + backward_nodes}")
print(f"\n  Meeting point: {bidir_path[forward_nodes]}")

# Compare with regular BFS
regular_path = nx.shortest_path(G, start, goal)
print(f"\nRegular BFS would explore more nodes typically.")
print(f"Bidirectional is especially faster on large graphs!")

## 4. Real-World Application: Social Network Analysis

Let's analyze a friend network to find connections between people.

In [ ]:
# Create a social network
social_network = nx.Graph()

# Add people and friendships
friendships = [
    ('Alice', 'Bob'), ('Alice', 'Carol'), ('Alice', 'David'),
    ('Bob', 'Eve'), ('Bob', 'Frank'),
    ('Carol', 'Grace'), ('Carol', 'Henry'),
    ('David', 'Ivan'),
    ('Eve', 'Jack'), ('Frank', 'Jack'),
    ('Grace', 'Kelly'), ('Henry', 'Kelly'),
    ('Ivan', 'Larry'), ('Jack', 'Larry'),
    ('Kelly', 'Mary'), ('Larry', 'Mary'),
]

social_network.add_edges_from(friendships)

# Visualize
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(social_network, seed=42, k=1.5)
nx.draw(social_network, pos, with_labels=True, 
        node_color='lightpink', node_size=2000,
        font_size=10, font_weight='bold',
        edge_color='gray', width=2, alpha=0.7)
plt.title('Social Network', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

# Find shortest path between two people
person1 = 'Alice'
person2 = 'Mary'

connection_path = nx.shortest_path(social_network, person1, person2)
print(f"\nConnection from {person1} to {person2}:")
print(f"  {' → '.join(connection_path)}")
print(f"  Degrees of separation: {len(connection_path) - 1}")

# Find all shortest paths
all_connection_paths = list(nx.all_shortest_paths(social_network, person1, person2))
print(f"\nAll shortest connections: {len(all_connection_paths)}")
for i, path in enumerate(all_connection_paths[:3], 1):  # Show first 3
    print(f"  {i}. {' → '.join(path)}")

In [ ]:
# Social network metrics
print("Social Network Analysis:")
print("=" * 60)

# Centrality: Who is most connected?
degree_centrality = nx.degree_centrality(social_network)
most_connected = max(degree_centrality.items(), key=lambda x: x[1])
print(f"Most connected person: {most_connected[0]} (centrality: {most_connected[1]:.3f})")

# Betweenness: Who is the best connector?
betweenness = nx.betweenness_centrality(social_network)
best_connector = max(betweenness.items(), key=lambda x: x[1])
print(f"Best connector: {best_connector[0]} (betweenness: {best_connector[1]:.3f})")

# Average shortest path
avg_path = nx.average_shortest_path_length(social_network)
print(f"\nAverage degrees of separation: {avg_path:.2f}")

# Diameter: Maximum shortest path
diameter = nx.diameter(social_network)
print(f"Network diameter: {diameter} (max degrees of separation)")

# Clustering: How interconnected are friend groups?
clustering = nx.average_clustering(social_network)
print(f"Average clustering coefficient: {clustering:.3f}")

print("\n💡 This is how LinkedIn shows '2nd degree connections'!")

## 5. Performance Comparison: All Algorithms

Let's compare all the search algorithms we've learned on a larger graph.

In [ ]:
# Create a larger random graph
n_nodes = 100
large_graph = nx.gnm_random_graph(n_nodes, n_nodes * 3, seed=42)

# Add random weights
for u, v in large_graph.edges():
    large_graph[u][v]['weight'] = np.random.randint(1, 20)

start_node = 0
goal_node = n_nodes - 1

print(f"Large Graph: {n_nodes} nodes, {large_graph.number_of_edges()} edges")
print(f"Finding path from node {start_node} to node {goal_node}...\n")

# Test different algorithms
results = []

# BFS (unweighted)
start_time = time.time()
try:
    bfs_path = nx.shortest_path(large_graph, start_node, goal_node)
    bfs_time = time.time() - start_time
    results.append(('BFS', len(bfs_path), bfs_time, 'N/A'))
except:
    results.append(('BFS', 'No path', 0, 'N/A'))

# Dijkstra
start_time = time.time()
try:
    dijkstra_path = nx.dijkstra_path(large_graph, start_node, goal_node, weight='weight')
    dijkstra_cost = nx.dijkstra_path_length(large_graph, start_node, goal_node, weight='weight')
    dijkstra_time = time.time() - start_time
    results.append(('Dijkstra', len(dijkstra_path), dijkstra_time, dijkstra_cost))
except:
    results.append(('Dijkstra', 'No path', 0, 'N/A'))

# A* (with zero heuristic = Dijkstra)
start_time = time.time()
try:
    astar_path = nx.astar_path(large_graph, start_node, goal_node, 
                               heuristic=lambda u, v: 0, weight='weight')
    astar_cost = nx.astar_path_length(large_graph, start_node, goal_node,
                                      heuristic=lambda u, v: 0, weight='weight')
    astar_time = time.time() - start_time
    results.append(('A*', len(astar_path), astar_time, astar_cost))
except:
    results.append(('A*', 'No path', 0, 'N/A'))

# Bidirectional
start_time = time.time()
try:
    bidir_path, _, _ = bidirectional_search(large_graph, start_node, goal_node)
    bidir_time = time.time() - start_time
    results.append(('Bidirectional', len(bidir_path) if bidir_path else 'No path', 
                   bidir_time, 'N/A'))
except:
    results.append(('Bidirectional', 'No path', 0, 'N/A'))

# Display results
print("Performance Comparison:")
print("=" * 70)
print(f"{'Algorithm':<15} {'Path Length':<15} {'Time (ms)':<15} {'Total Cost'}")
print("-" * 70)
for algo, length, time_taken, cost in results:
    print(f"{algo:<15} {str(length):<15} {time_taken*1000:<15.2f} {cost}")

print("\n💡 For weighted graphs, Dijkstra/A* find optimal paths!")
print("💡 Bidirectional is faster on large graphs with long paths!")

## 6. Bringing It All Together: Summary

### What We've Learned This Week:

#### Lab 1: Uninformed Search
- **DFS**: Deep exploration, memory efficient
- **BFS**: Level-by-level, finds shortest path (steps)

#### Lab 2: Informed Search  
- **UCS**: Cost-aware, finds optimal path (cost)
- **Greedy**: Fast, follows heuristic
- **A***: Optimal + efficient (the best!)

#### Lab 3: Adversarial Search
- **Minimax**: Optimal strategy against opponent
- **Alpha-Beta**: Pruning for efficiency

#### Lab 4: Real-World Applications
- **NetworkX**: Professional graph library
- **Bidirectional**: Search from both ends
- **Applications**: Social networks, routing, analysis

### Algorithm Selection Guide:

| Problem Type | Best Algorithm | Why |
|--------------|----------------|-----|
| Unweighted graph | BFS | Finds shortest path (steps) |
| Weighted graph | A* or Dijkstra | Finds optimal path (cost) |
| Very large graph | Bidirectional | Explores fewer nodes |
| Game playing | Alpha-Beta | Optimal with pruning |
| Need any solution fast | DFS or Greedy | Quick, not optimal |
| Have good heuristic | A* | Best performance |

### Key Insights:

1. **Problem Representation Matters**: How you model the problem affects which algorithms work well

2. **Heuristics Are Powerful**: A good heuristic can speed up search dramatically

3. **Trade-offs Everywhere**: Speed vs optimality, memory vs time

4. **Use Libraries**: NetworkX, scikit-learn, etc. provide optimized implementations

5. **Search Is Everywhere**: GPS, games, AI planning, network routing, recommendation systems

### Performance Summary:

```
Algorithm       Time Complexity    Space Complexity   Optimal?   Complete?
────────────────────────────────────────────────────────────────────────────
BFS             O(b^d)            O(b^d)             Yes*       Yes
DFS             O(b^m)            O(bm)              No         Yes**
UCS             O(b^(1+⌊C*/ε⌋))   O(b^(1+⌊C*/ε⌋))   Yes        Yes
Greedy          O(b^m)            O(b^m)             No         No
A*              O(b^d)            O(b^d)             Yes***     Yes
Alpha-Beta      O(b^(d/2))        O(bd)              Yes        Yes
Bidirectional   O(b^(d/2))        O(b^(d/2))         Yes*       Yes

* Optimal for unweighted graphs
** Complete in finite spaces
*** Optimal if heuristic is admissible
```

## 7. Final Challenges

Test your knowledge with these exercises:

In [ ]:
# Challenge 1: Find the most important person in a network
# Hint: Use centrality measures

# Challenge 2: Implement iterative deepening DFS
# Combines benefits of DFS (memory) and BFS (optimal)

# Challenge 3: Create a route planner with multiple stops
# This is the Traveling Salesman Problem!

# Challenge 4: Build a recommendation system
# "People who know Alice also know..."

# Challenge 5: Implement MCTS (Monte Carlo Tree Search)
# Used in AlphaGo!

print("Ready for these challenges? You have all the tools you need!")
print("\nCongratulations on completing Week 1: Search! 🎉")
print("\nYou've learned the fundamental algorithms that power:")
print("  📍 Google Maps navigation")
print("  ♟️  Chess engines like Stockfish")
print("  🤝 LinkedIn connection suggestions")
print("  🎮 Video game AI")
print("  🌐 Internet routing protocols")
print("\nNext up: Week 2 - Knowledge Representation!")

## Resources for Further Learning

### Books
- Russell & Norvig: "Artificial Intelligence: A Modern Approach" (Chapter 3-5)
- Cormen et al.: "Introduction to Algorithms" (Graph algorithms)

### Online Resources
- [NetworkX Documentation](https://networkx.org/documentation/stable/)
- [Pathfinding Visualizer](https://qiao.github.io/PathFinding.js/visual/)
- [Red Blob Games: A* Tutorial](https://www.redblobgames.com/pathfinding/a-star/introduction.html)

### Papers
- "A Formal Basis for the Heuristic Determination of Minimum Cost Paths" (Hart et al., 1968) - Original A* paper
- "Mastering the Game of Go with Deep Neural Networks" (Silver et al., 2016) - AlphaGo

### Practice
- LeetCode: Graph problems
- Project Euler: Pathfinding puzzles
- Kaggle: Network analysis competitions

---

**Congratulations!** You've completed an intensive introduction to search algorithms. These concepts form the foundation for much of AI, and you'll see them again and again as you continue your journey.

Keep practicing, keep exploring, and keep building! 🚀